<a href="https://colab.research.google.com/github/IlyaKuprik/skolkovo-hack/blob/main/preprocessing_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Processing Data

## Загрузка


In [63]:
data_candidates_work_places = pd.read_csv('/content/drive/MyDrive/hack/data/data_candidates_work_places.csv', sep=';',
                                names = ['CandidateId', 'Position','FromYear', 'FromMonth', 'ToYear', 'ToMonth'])
data_jobs = pd.read_csv('/content/drive/MyDrive/hack/data/data_jobs.csv', sep=';',
                                names = ['JobId', 'Status','Name', 'Region', 'Description'])
data_candidates = pd.read_csv('/content/drive/MyDrive/hack/data/data_candidates.csv', sep=';',
                                names = ['CandidateId', 'Position','Sex', 'Citizenship', 'Age',
                                'Salary', 'Langs', 'DriverLicense', 'Subway', 'Skills', 'Employment', 'Shedule',                                           
                                 'CandidateRegion', 'DateCreated', 'JobId', 'CandidateStatusId', 'Status'])
data_candidates_education = pd.read_csv('/content/drive/MyDrive/hack/data/data_candidates_education.csv', sep=';',
                                names = ['CandidateId', 'University', 'Faculty', 'GraduateYear'])

## Обработка таблицы

In [65]:
import re

#Удаление html разметки в поле Description
CLEANR = re.compile('<.*?>|&([a-z0-9]+|#[0-9]{1,6}|#x[0-9a-f]{1,6});|-|/|;|_|•')
 
def cleanhtml(raw_html):
    if type(raw_html) == type(''):
        return re.sub(CLEANR, '', raw_html)
    return raw_html
 
data_jobs.Description = data_jobs.Description.apply(cleanhtml)

In [ ]:
cols_val = {str(i): 'last' for i in data_candidates.columns}

In [70]:
#Группировка и замена Status последним изменение статуса
cols_val = {str(i): 'last' for i in data_candidates.columns}
grouped_with_last_status = data_candidates.sort_values(by = 'DateCreated').loc[:,data_candidates.columns]\
.groupby(by = ['CandidateId','JobId'], axis = 0, sort = False, as_index = False).agg( cols_val)

### Создание меток 

In [73]:
#Инициализация начальных меток
target_map = {"Самоотказ РЦ": 1, "Не отвечает": 0.1, "Отказ РЦ": 0, "Думает РЦ": 0.3, "Не пришел": 0.7, "Приглашен": 0.85, "Отклонен": 0.5, "Не пришёл": 0.8, 
  "Назначено собеседование в ресторан": 0.8, "Телефонное интервью": 0.7, "Пришел": 0.5, "Обзвон": 0.6, "ОТКАЗ СЦ": 0.3, "Отказ кандидата": 1, 
  "Отклик": 0.9, "Думает МП":0.4, "Расторгнул договор": 0.15, "Лендинг": 0.6, "Резерв": 0.5, "Отказ": 0.3, 'Вышел на линию': 0.9,
  "Принят на работу": 0.95, "Оформлен": 0.9, "Неактуально": 0, "Отказ компании": 0, "Перезаписался": 0.4, "Отказался/Нашел работу": 0.3,
  "Заключил договор": 0.8, "Отправлен на СБ": 0.85, "Интервью с HR": 0.85, "Самоотказ МП": 0.5, "Недозвон": 0.2, "Отказ по итогам скрининга резюме": 0,
  "Не прошёл СБ": 0.5, "Отказ HR": 0, "Резерв РЦ": 0.2, "Не пришел (повторно)": 0.6, "Интервью": 0.8, "Отказ ВВСС (другое)": 0.2, "Отказ СБ": 0.65}

In [74]:
targets = grouped_with_last_status['Status'].apply(lambda x: target_map[x] if x in target_map else np.NaN) ## Заполнили NA 
grouped_with_ids_target = grouped_with_last_status.loc[:, ['CandidateId', 'JobId',]]
grouped_with_ids_target['target'] = targets

In [81]:
#Инициализация штрафов

number_fields = ['Age', 'Salary']
string_fields = ['Position', 'Citizenship', 'Langs',\
                  'Skills', 'Employment', 'Shedule', 'CandidateRegion']
fines_vector = [1/9]*9 

In [82]:

fine_mask = grouped_with_last_status.copy()
fine_mask[number_fields] = grouped_with_last_status[number_fields].apply(lambda x: x == 0)
fine_mask[string_fields] = grouped_with_last_status[string_fields].apply(lambda x: x.isna())

total_fine = fine_mask[number_fields + string_fields].apply(lambda x: 1 - (np.array(x) * np.array(fines_vector)).sum(), axis = 1)
total_target = targets*total_fine
grouped_with_ids_target['target'] = total_target

### Driver license

In [83]:
def del_sep(text):
    if type(text) == type(''):
        return ''.join(text.split(','))
    return text

data_candidates.DriverLicense = data_candidates.DriverLicense.apply(del_sep)

### Skills

In [84]:
def del_stick(text):
    if type(text) == type(''):
        return ''.join(text.split('||'))
    return text


data_candidates.Skills = data_candidates.Skills.apply(del_stick)

### Обработка текста

In [ ]:
candidates_work_places_concat = data_candidates_work_places.drop_duplicates(['CandidateId', 'Position'])
candidates_work_places_concat.Position = candidates_work_places_concat.Position.add(' ')
candidates_work_places_concat = candidates_work_places_concat.groupby('CandidateId', as_index=False).agg(Position_last_work = ('Position' , 'sum'))
candidates_work_places_concat

In [91]:
grouped_table_candidates_features = grouped_with_last_status.drop(columns = ['Status', 'CandidateStatusId'])

In [97]:
merged_candidates_features = grouped_table_candidates_features\
    .merge(candidates_work_places_concat, on='CandidateId', how='left')\
    .merge(data_candidates_education.drop_duplicates('CandidateId'), on='CandidateId', how='left')

In [99]:
#Удаление наблюдений с одинаковыми полями
merged_candidates_features = merged_candidates_features.drop_duplicates(['Position', 'Citizenship', 'DriverLicense',\
                        'Employment', 'Shedule', 'CandidateRegion',\
                        'Position_last_work', 'University', 'Faculty' ])

In [108]:

merged_candidates_features.index = [merged_candidates_features.CandidateId, merged_candidates_features.JobId]
part_candidates_features = merged_candidates_features.drop(columns=['JobId', 'CandidateId', 'Salary', 'Subway',
                          'Age', 'GraduateYear', 'DateCreated'])
part_candidates_features.Sex = part_candidates_features.Sex\
    .where(~(part_candidates_features.Sex == 2), other='мужчина')\
    .where(~(part_candidates_features.Sex == 1), other='женщина')\
    .where(~(part_candidates_features.Sex == 0), other='неопределен')
part_candidates_features = part_candidates_features.fillna('NaN').astype(str).add('. ').sum(axis=1).reset_index()
candidates_tokens = part_candidates_features

status_jobs = pd.DataFrame([[1, 'открытая'], [2, 'закрытая'], [3, 'приостановленная'], [4, 'отменена'], [5, 'черновик']],
                          columns = ['Status', 'name_status'])
new_data_jobs = data_jobs\
    .merge(status_jobs, on='Status')
new_data_jobs.index = new_data_jobs.JobId
jobs_tokens = new_data_jobs\
    .drop(columns=['Status', 'JobId'])\
    .fillna('NaN')\
    .astype(str)\
    .add('. ')\
    .sum(axis=1)\
    .reset_index()
 

job_user_tokens_with_id = candidates_tokens\
    .merge(jobs_tokens, on='JobId')
 

In [117]:
merged_descriptions_targets_ids = job_user_tokens_with_id.merge(grouped_with_ids_target, on = ['JobId', 'CandidateId'], how = 'inner')
merged_descriptions_targets_ids.columns = ["CandidateId", "JobId", "Candidate_descr", "Jobs_descr", "target"]
#Удаляем пустые поля в target (не размечанное)
merged_descriptions_targets_ids = merged_descriptions_targets_ids[~(merged_descriptions_targets_ids.target.isna())]
merged_descriptions_targets_ids.drop(columns = ['CandidateId', 'JobId'], inplace = True)
merged_descriptions_targets_ids = merged_descriptions_targets_ids.drop_duplicates(['Candidate_descr', 'Jobs_descr'])

In [118]:
merged_descriptions_targets_ids.to_csv('preprocess_data.csv')

## Загрузка теста


In [35]:
test_candidates = pd.read_csv('/content/drive/MyDrive/hack/data/test/test_candidates.csv', sep = ';')

In [36]:
test_jobs = pd.read_csv('/content/drive/MyDrive/hack/data/test/test_jobs.csv', sep = ';',\
                        names = ['JobId', 'Status','Name', 'Region', 'Description', 'x', 'y','z'])
test_jobs.drop(columns = ['x', 'y','z'], inplace = True)

In [37]:
test_candidates_education = pd.read_csv('/content/drive/MyDrive/hack/data/test/test_candidates_education.csv', sep=';')

In [38]:
test_candidates_workplaces = pd.read_csv('/content/drive/MyDrive/hack/data/test/test_candidates_workplaces.csv', sep = ';')

## Обработка test_jobs

In [119]:
import re
 
CLEANR = re.compile('<.*?>|&([a-z0-9]+|#[0-9]{1,6}|#x[0-9a-f]{1,6});|-|/|;|_|•')
 
def cleanhtml(raw_html):
    if type(raw_html) == type(''):
        return re.sub(CLEANR, '', raw_html)
    return raw_html
 
test_jobs.Description = test_jobs.Description.apply(cleanhtml)

In [120]:
status_jobs = pd.DataFrame([[1, 'открытая'], [2, 'закрытая'], [3, 'приостановленная'], [4, 'отменена'], [5, 'черновик']],
                          columns = ['Status', 'name_status'])
new_data_jobs = test_jobs\
    .merge(status_jobs, on='Status')
new_data_jobs.index = new_data_jobs.JobId
job_user_tokens1 = new_data_jobs\
    .drop(columns=['Status', 'JobId'])\
    .fillna('NaN')\
    .astype(str)\
    .add('. ')\
    .sum(axis=1)\
    .reset_index()

## Обработка бывшего места работы

In [ ]:
candidates_work_places_concat = test_candidates_workplaces.drop_duplicates(['CandidateId', 'Position'])
candidates_work_places_concat.Position = candidates_work_places_concat.Position.add(' ')
candidates_work_places_concat = candidates_work_places_concat.groupby('CandidateId', as_index=False).agg({'Position' : 'sum'})
candidates_work_places_concat

## Обработка кандидата

In [123]:
def del_sep(text):
    if type(text) == type(''):
        return ''.join(text.split(','))
    return text
test_candidates.DriverLicense = test_candidates.DriverLicense.apply(del_sep)

In [124]:
def del_stick(text):
    if type(text) == type(''):
        return ''.join(text.split('||'))
    return text
test_candidates.Skills = test_candidates.Skills.apply(del_stick)

In [125]:
part_test_candidates_features = test_candidates\
    .merge(candidates_work_places_concat, on='CandidateId', how='left')\
    .merge(test_candidates_education.drop_duplicates('CandidateId'), on='CandidateId', how='left')
part_test_candidates_features.index = part_test_candidates_features.CandidateId
df = part_test_candidates_features.drop(columns=['CandidateId', 'Salary', 'Subway',
                          'Age', 'GraduateYear'])
df.Sex = df.Sex\
    .where(~(df.Sex == 2), other='мужчина')\
    .where(~(df.Sex == 1), other='женщина')\
    .where(~(df.Sex == 0), other='неопределен')
df = df.fillna('NaN').astype(str).add('. ').sum(axis=1).reset_index()

job_user_tokens2 = df

In [131]:
job_user_tokens2 = job_user_tokens2.rename(columns = {0: 'Candidate_descr'})
job_user_tokens1 = job_user_tokens1.rename(columns = {0: 'Jobs_descr'})

In [132]:
test_data_merged = job_user_tokens2.merge(job_user_tokens1, how = 'cross')
test_data_merged.drop(columns = ['CandidateId', 'JobId'], inplace = True)

In [133]:
test_data_merged

,Candidate_descr,Jobs_descr
0,специалист отдела парковочного комплекса. мужч...,Водитель автомобиля большегруза с прицепом . С...
1,специалист отдела парковочного комплекса. мужч...,водитель на своем авто . Москва. Начался сезон...
2,специалист отдела парковочного комплекса. мужч...,Водитель на своем авто. Москва. Стаж вождения ...
3,специалист отдела парковочного комплекса. мужч...,Водитель. Санкт-Петербург. Обязанности:Качеств...
4,специалист отдела парковочного комплекса. мужч...,водитель на личном авто. Москва. ищем курьера ...
...,...,...
1995,Менеджер по продажам. мужчина. Россия. русски...,Водитель-экспедитор 1/ Водитель. Москва. Обяза...
1996,Менеджер по продажам. мужчина. Россия. русски...,водитель курьер на авто компании. Москва. води...
1997,Менеджер по продажам. мужчина. Россия. русски...,Менеджер по продажам. Санкт-Петербург. Обязанн...
1998,Менеджер по продажам. мужчина. Россия. русски...,Водитель-экспедитор. Москва. ЗАДАЧИ: Доставка...


In [59]:
test_data_merged.to_csv('test_data_merged.csv')